In [4]:
from hazm import *
import openpyxl
from pathlib import Path
import collections
import math
import itertools
import pickle

In [5]:

N_documents = 0
normalizer = Normalizer()
stemmer = Stemmer()
tokens = []
doc_vectors = {}

In [6]:
punctuations = [':', '،', '.', ')', '(', '}', '{', '؟', '!', '-', '/', '؛', '#', '*', '\n', '\"',
                ']', '[', '«', '»', '٪', '+', '٠', '\\', '\"', '_', '\'']



In [22]:
%%time

xlsx_file = Path('../IR_Spring2021_ph12_7k.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file)
sheet = wb_obj.active
for row in sheet.iter_rows():
    each_doc_vector = {}
    line = row[1].value
    if row[0].value == 'id':
        continue
    N_documents += 1
    line = normalizer.normalize(line)
    for p in punctuations:
        if p in line:
            line = line.replace(p, " ")
    extracted_tokens = word_tokenize(line)
    stem_tokens = []
    for t in extracted_tokens:
        stem_tokens.append(stemmer.stem(t))

    term_doc = []
    for w in stem_tokens:
        term_doc.append({'term': w, 'doc_id': int(row[0].value)})
    tokens.extend(term_doc)
    
    for t in stem_tokens:
        each_doc_vector[t] = each_doc_vector.get(t, 0) + 1
    doc_vectors[int(row[0].value)] = collections.OrderedDict(sorted(each_doc_vector.items()))


Wall time: 18.2 s


In [23]:
%%time
dictionary = {}
inverted_index = {}
idf = {}
doc_term_frq = []
for t in tokens:
    word = t['term']
    doc_id = t['doc_id']
    
    frq = dictionary.get(word, 0)
    dictionary[word] = frq + 1

    doc_list = inverted_index.get(word, [])
    doc_list.append(doc_id)
    inverted_index[word] = doc_list
    
    

Wall time: 1.95 s


In [24]:
%%time
# remove stop words
frq_sorted_dic = {k: v for k, v in sorted(dictionary.items(), key=lambda item: item[1])}
stop_words = dict(itertools.islice(reversed(frq_sorted_dic.items()), 20))
for s in stop_words:
    dictionary.pop(s)
    inverted_index.pop(s)

Wall time: 21.9 ms


In [25]:
%%time

inver_inex_dup = inverted_index.copy()
for i in inverted_index:
    inverted_index[i] = list(dict.fromkeys(inverted_index[i]))
inverted_index = collections.OrderedDict(sorted(inverted_index.items()))
dictionary = collections.OrderedDict(sorted(dictionary.items()))

print(inverted_index.keys())

odict_keys(['', '&', '&NID=', '&pageid=', ',', ',everywhere', ';', ';A', '<http', '<https', '=', '>', '?', '@HAMAYESHSALAMATVAZENDEGI', '@Motamed', '@gmail', '@isna', '@motahari', '@student', '@unesco', '@unescocps', 'A', 'A,C,D,E', 'AA', 'AB', 'AC', 'ACA', 'ACE', 'ACE۲', 'ACL', 'AC۲', 'AD', 'ADSL', 'AE', 'AF', 'AFC', 'AFM', 'AHF', 'AI', 'AIDS', 'AIIB', 'AMD', 'AMH', 'AMP', 'ANDEROONشد', 'ANOC', 'APKCombo', 'APKpure', 'APM', 'APP', 'APT', 'ARJ', 'ASFR', 'ASMRاس', 'ASR', 'ASTAR', 'ATM', 'ATR', 'About', 'Academic', 'Academy', 'Added', 'Advanced', 'Advances', 'AfC', 'Age', 'Agent', 'Aix', 'Analysis', 'Annals', 'Antiblock', 'App', 'Arabic', 'Archive', 'Associated', 'AstraZeneca', 'Attenuated', 'Awakend', 'Awakened', 'A۱', 'A۱C', 'A۲', 'A۳', 'A۶', 'A۷', 'A۸', 'A۹', 'B', 'BA', 'BAK', 'BBIBP', 'BE', 'BI', 'BICU', 'BMI', 'BMJ', 'BMT', 'BMW', 'BNT', 'BTS', 'Based', 'Baseline', 'Batch', 'Belt', 'BioNTech', 'Bit', 'Bjelovar', 'Blower', 'Bortezomib', 'Braskem', 'Break', 'Brown', 'Business', 'B۰', 

In [26]:
%%time

for term in inverted_index:
    df = len(inverted_index[term])
    idf[term] = math.log(N_documents / df, 10)
    print(idf[term])

0.7537828803170339
3.5440680443502752
3.5440680443502752
3.8450980400142565
2.5663444390614276
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.5440680443502752
3.1461280356782377
2.6690067809585756
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.3679767852945943
2.1730001820785394
3.8450980400142565
1.947470948723815
2.6146491186359824
2.1549019599857426
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
2.2653144433974464
2.9999999999999996
2.1549019599857426
1.8813102126687014
2.6690067809585756
3.5440680443502752
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.84509804001

3.3679767852945943
1.6491983876050231
1.6748363246192992
2.1375278639163207
2.232314183294521
1.8407766662316138
3.8450980400142565
1.9586073148417746
3.8450980400142565
2.8037053548560316
3.243038048686294
3.8450980400142565
3.243038048686294
3.8450980400142565
3.8450980400142565
2.6409780573583315
3.5440680443502752
3.3679767852945943
3.8450980400142565
3.8450980400142565
2.413734275855269
3.5440680443502752
3.3679767852945943
2.9999999999999996
2.6690067809585756
3.1461280356782377
2.9999999999999996
2.9420080530223127
3.8450980400142565
3.1461280356782377
3.8450980400142565
3.5440680443502752
2.9420080530223127
3.8450980400142565
3.243038048686294
2.8037053548560316
3.3679767852945943
2.6989700043360183
3.8450980400142565
3.8450980400142565
3.1461280356782377
3.5440680443502752
3.8450980400142565
3.3679767852945943
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
2.232314183294521
3.8450980400142565
3.3679767852945943
3.8450980400142565
2.2653144433974464

3.8450980400142565
3.8450980400142565
2.9999999999999996
3.3679767852945943
3.8450980400142565
1.402618270949808
3.5440680443502752
3.1461280356782377
3.243038048686294
3.3679767852945943
3.3679767852945943
3.8450980400142565
3.8450980400142565
3.8450980400142565
2.8450980400142565
3.1461280356782377
1.30602194122148
3.8450980400142565
3.3679767852945943
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.066946789630613
2.9420080530223127
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.243038048686294
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
1.70837747285785
2.8908555305749317
2.9420080530223127
3.3679767852945943
3.3679767852945943
3.8450980400142565
3.5440680443502752
2.6989700043360183
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.3679767852945943
3.5440680443502752
3.8450980400142565
3.8450980400142565
3

3.8450980400142565
3.8450980400142565
3.066946789630613
2.8037053548560316
3.8450980400142565
2.104735350520013
2.6690067809585756
2.0816700464513196
2.232314183294521
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.5440680443502752
2.8037053548560316
3.5440680443502752
2.9999999999999996
2.73115468770742
3.5440680443502752
3.8450980400142565
2.6690067809585756
3.8450980400142565
3.243038048686294
3.1461280356782377
3.3679767852945943
2.8908555305749317
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.3679767852945943
3.5440680443502752
2.73115468770742
3.8450980400142565
3.8450980400142565
1.9758663202832807
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.3679767852945943
3.8450980400142565
2.9999999999999996
3.5440680443502752
2.9999999999999996
3.5440680443502752
3.1461280356782377
2.3399480616943507
3.8450980400142565
3.8450980400142565
2.0190232373134305
3.8450980400142565
3.066946789630613
1.2912150133703824
2.4301246920434387
2.

3.8450980400142565
3.243038048686294
3.243038048686294
3.8450980400142565
3.5440680443502752
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.243038048686294
3.8450980400142565
2.8450980400142565
2.6146491186359824
3.5440680443502752
3.8450980400142565
3.5440680443502752
3.3679767852945943
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.3679767852945943
3.5440680443502752
2.8037053548560316
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.066946789630613
3.8450980400142565
3.5440680443502752
3.5440680443502752
3.8450980400142565
2.9999999999999996
3.8450980400142565
3.8450980400142565
3.8450980400142565
2.8450980400142565
3.066946789630613
3.243038048686294
2.397940008672037
3.8450980400142565
3.5440680443502752
2.9999999999999996
3.1461280356782377
3.8450980400142565
2.9999999999999996
3.5440680443502752
3.8450980400142565
2.8908555305749317


...

3.3679767852945943
2.8908555305749317
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
2.483370203996664
3.1461280356782377
3.3679767852945943
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.243038048686294
3.8450980400142565
2.4648867983026506
3.5440680443502752
3.5440680443502752
3.5440680443502752
3.8450980400142565
3.3679767852945943
2.765916793966632
3.066946789630613
3.5440680443502752
2.6989700043360183
3.5440680443502752
2.9999999999999996
3.5440680443502752
3.3679767852945943
3.8450980400142565
3.8450980400142565
3.243038048686294
3.3679767852945943
2.9420080530223127
3.8450980400142565
1.6690067809585756
3.1461280356782377
3.8450980400142565
3.5440680443502752
2.8908555305749317
3.8450980400142565
3.5440680443502752
3.5440680443502752
1.663254452069484
3.8450980400142565
3.5440680443502752
1.9006153678640882
3.8450980400142565
3.243038048686294
3.8450980400142565
3.066946789630613
3.8450980400142565
2

2.8908555305749317
3.3679767852945943
2.765916793966632
3.8450980400142565
3.8450980400142565
1.9877655435829882
2.4648867983026506
3.8450980400142565
3.8450980400142565
3.5440680443502752
2.9999999999999996
3.8450980400142565
3.8450980400142565
3.1461280356782377
2.9999999999999996
3.3679767852945943
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.3679767852945943
3.8450980400142565
3.8450980400142565
3.3679767852945943
3.3679767852945943
3.5440680443502752
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
0.7492315865357141
2.483370203996664
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.3679767852945943
1.9366130211356067
3.8450980400142565
3.8450980400142565
3.5440680443502752
2.5440680443502752
3.5440680443502752
3.8450980400142565
3.5440680443502752
3.066946789630613
3.8450980400142565
2.9999999999999996
3.8450980400142565
3.066946789630613
2.447158031342219
2.301029995663981
3.8450980400142565
2.9999999999999996

3.3679767852945943
3.8450980400142565
3.8450980400142565
3.3679767852945943
2.0527063505160026
3.3679767852945943
2.6690067809585756
3.243038048686294
2.765916793966632
2.5440680443502752
3.243038048686294
3.8450980400142565
3.243038048686294
3.8450980400142565
3.5440680443502752
1.5506318138526638
3.8450980400142565
3.3679767852945943
3.8450980400142565
2.3399480616943507
3.8450980400142565
3.5440680443502752
2.1823402083326826
2.8037053548560316
3.8450980400142565
3.5440680443502752
3.5440680443502752
2.112704280191288
3.066946789630613
3.8450980400142565
3.8450980400142565
1.3399480616943509
2.6409780573583315
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
2.9999999999999996
3.243038048686294
3.5440680443502752
2.8037053548560316
3.8450980400142565
3.243038048686294
3.5440680443502752
3.066946789630613
3.1461280356782377
3.8450980400142565
3.8450980400142565
3.8450980400142565
2.288795539246969
3.3

3.5440680443502752
3.8450980400142565
3.5440680443502752
3.5440680443502752
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.1461280356782377
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400

1.4541629329108776
2.6409780573583315
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.5440680443502752
3.5440680443502752
3.5440680443502752
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.5440680443502752
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400142565
3.8450980400

# Index Elimination

In [27]:
%%time

index_elimination_model = {}
for term in inverted_index.keys():
    index_elimination_model[term] = {}
    for doc in inverted_index[term]:
        index_elimination_model[term][doc] = inver_inex_dup[term].count(doc)

# print(index_elimination_model['بود'])

Wall time: 18.8 s


### Champion List

In [28]:
%%time

champion_size = 150
champion_list = {}
for term in index_elimination_model.keys():
    document_dict = index_elimination_model[term]
    sorted_result = {k: v for k, v in sorted(document_dict.items(), key=lambda item: item[1])}
    sorted_result = dict(itertools.islice(reversed(sorted_result.items()), champion_size))
    champion_list[term] = collections.OrderedDict(sorted(sorted_result.items()))
# print(champion_list['بود'])

Wall time: 498 ms


### File Writting

In [29]:
%%time
file = open('stop_words.obj', 'wb') 
pickle.dump(stop_words, file)
file.close()

file = open('index_elimination.obj', 'wb') 
pickle.dump(index_elimination_model, file)
file.close()

file = open('idf.obj', 'wb') 
pickle.dump(idf, file)
file.close()

file = open('doc_vec.obj', 'wb') 
pickle.dump(doc_vectors, file)
file.close()

file = open('champion.obj', 'wb') 
pickle.dump(champion_list, file)
file.close()

Wall time: 654 ms


### File Reading

In [7]:
readed = open('stop_words.obj', 'rb')
stop_words = pickle.load(readed)

readed = open('index_elimination.obj', 'rb')
index_elimination_model = pickle.load(readed)

readed = open('idf.obj', 'rb')
idf = pickle.load(readed)

readed = open('champion.obj', 'rb')
champion_list = pickle.load(readed)

readed = open('doc_vec.obj', 'rb')
doc_vectors = pickle.load(readed)


### Query

In [25]:
%%time
query = input("Serach: ")
query = normalizer.normalize(query)
for p in punctuations:
    if p in query:
        query = query.replace(p, " ")
extracted_tokens = word_tokenize(query)
stem_tokens = []
for t in extracted_tokens:
    stem_tokens.append(stemmer.stem(t))
for s in stop_words:
    if s in stem_tokens:
        for t in stem_tokens:
            if t == s:
                stem_tokens.remove(t)

Serach: پلی‌آف لیگ قهرمانان آسیا
Wall time: 12.4 s


In [26]:
%%time
query_frq = {}
for t in stem_tokens:
    query_frq[t] = query_frq.get(t, 0) + 1
print(query_frq)

{'پلی\u200cآف': 1, 'لیگ': 1, 'قهرمان': 1, 'آسیا': 1}
Wall time: 98.6 ms


# Similarity

### No Champion List Results

In [27]:
result = {}

for term in query_frq.keys():
    wq = 1+math.log(query_frq[term], 10)
    for doc in index_elimination_model[term].keys():
        doc_tf = index_elimination_model[term].get(doc, 0)
        wd = (1+math.log(doc_tf, 10))*(idf[term])
        result[doc] = result.get(doc, 0) + (wd*wq)
        
for doc in result.keys():
    l2_length = 0
    for t in doc_vectors[doc].values():
        l2_length += (1+math.log(t, 10))**2
    l2_length = math.sqrt(l2_length)
    result[doc] = result.get(doc, 0)/l2_length
print(result)

{70: 0.3418453526859365, 73: 0.4143616099349263, 154: 0.6969218459640325, 252: 0.32524176812594774, 321: 0.4032903279518231, 394: 0.7154569883925582, 632: 0.7475086558119229, 1094: 0.45442736461975647, 1583: 0.566468735292778, 1595: 0.35072094482705757, 1596: 0.7794424768671278, 1597: 0.44460523422047177, 1672: 0.4132392380232029, 2: 0.1774443214902801, 14: 0.16546035623214136, 21: 0.2227503444996783, 38: 0.12551564407851593, 44: 0.14247684105821207, 45: 0.09811983573102495, 47: 0.42222658231877747, 48: 0.2741296979448253, 50: 0.1557229354143215, 52: 0.15712624990337623, 57: 0.2515042450232563, 60: 0.10935193960914268, 66: 0.3697341365007185, 72: 0.06448635275088596, 81: 0.04089652195811852, 86: 0.1433600140793261, 92: 0.2532189089443874, 96: 0.096676987362651, 97: 0.11755918715614423, 100: 0.10364197456249732, 103: 0.46895366495731244, 107: 0.09344310517260343, 108: 0.11976313688535861, 110: 0.24257194582909425, 111: 0.09268557576728334, 113: 0.1221675840671567, 116: 0.128413254300363

### Champion List Results

In [23]:
%%time

result = {}

for term in query_frq.keys():
    wq = 1+math.log(query_frq[term], 10)
    for doc in champion_list[term].keys():
        doc_tf = index_elimination_model[term].get(doc, 0)
        wd = (1+math.log(doc_tf, 10))*(idf[term])
        result[doc] = result.get(doc, 0) + (wd*wq)
        
for doc in result.keys():
    l2_length = 0
    for t in doc_vectors[doc].values():
        l2_length += (1+math.log(t, 10))**2
    l2_length = math.sqrt(l2_length)
    result[doc] = result.get(doc, 0)/l2_length
print(result)

{1186: 0.12809237853751684, 1857: 0.14504307833071875, 1937: 0.11538718910447648, 2024: 0.10380108292157011, 2039: 0.0889138507303472, 2069: 0.12589754946113912, 2263: 0.1258262951933158, 2483: 0.1173635243442009, 2704: 0.0642187060794893, 2735: 0.08197500428919592, 2736: 0.1359331140149117, 2759: 0.1407217881771058, 2787: 0.07148819568227577, 2797: 0.07167955691088122, 2932: 0.1556071098850886, 2944: 0.14797810049299, 3015: 0.11190530993444098, 3099: 0.06485578502071145, 3103: 0.14217048976252508, 3199: 0.186167909599743, 3210: 0.11328572472014024, 3352: 0.12780931377039645, 3576: 0.06885424906626347, 3661: 0.11819646389506072, 3665: 0.11819646389506072, 3716: 0.1298689763261885, 3750: 0.0889138507303472, 3775: 0.12589754946113912, 4126: 0.07700097034880317, 4322: 0.16553519687529064, 4554: 0.061825040836454105, 4560: 0.09686220127099411, 4765: 0.1325854434118337, 4768: 0.10766649563251364, 4807: 0.11136778198711568, 4809: 0.10703646958724103, 4816: 0.08447927557505165, 4828: 0.061152

# Results

In [28]:
K = 50

### Normal Sort

In [13]:
xlsx_file = Path('../IR_Spring2021_ph12_7k.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file)
sheet = wb_obj.active
sorted_result = {k: v for k, v in sorted(result.items(), key=lambda item: item[1])}
sorted_result = dict(itertools.islice(reversed(sorted_result.items()), K))
for doc in sorted_result.keys():
    for row in sheet.iter_rows():
        if row[0].value == 'id':
            continue
        if int(row[0].value) == doc:
            print(f"Document {doc} :\n{row[2].value}", end='\n____________________________________________\n')


Document 4514 :
https://www.isna.ir/news/98022915417/عرضه-آخرین-دستاوردهای-استارتاپ-های-حوزه-اقتصاد-دیجیتال-در-نمایشگاه
____________________________________________
Document 6641 :
https://www.isna.ir/news/98030200713/برگزاری-پاویون-شرکت-های-دانش-بنیان-در-دو-نمایشگاه-حوزه-سلامت
____________________________________________
Document 6637 :
https://www.isna.ir/news/98030200713/برگزاری-پاویون-شرکت-های-دانش-بنیان-در-دو-نمایشگاه-حوزه-سلامت
____________________________________________
Document 6635 :
https://www.isna.ir/news/98030200713/برگزاری-پاویون-شرکت-های-دانش-بنیان-در-دو-نمایشگاه-حوزه-سلامت
____________________________________________
Document 4528 :
https://www.isna.ir/news/98030200713/برگزاری-پاویون-شرکت-های-دانش-بنیان-در-دو-نمایشگاه-حوزه-سلامت
____________________________________________
Document 672 :
https://www.isna.ir/news/99082013716/صاحب-سوت-طلایی-فیلا-درگذشت
____________________________________________
Document 3374 :
https://www.isna.ir/news/99022417557/برپایی-پاویون-شرکت-های

### Heap

In [29]:
%%time
import heapq as hq
# List to hold values from dictionary
heap_dict=[]

# extract the values from dictionary
for i in result.values():
    heap_dict.append(i)
    
# heapify the values
hq._heapify_max(heap_dict)   

sorted_result = []
while len(sorted_result) < K:
    value = hq._heappop_max(heap_dict)
    for key,val in result.items():
        if value == val:
            sorted_result.append(key)
            break
              
print(sorted_result)

[1596, 632, 394, 154, 1583, 1012, 667, 1584, 1584, 1048, 832, 1020, 1244, 103, 615, 156, 1654, 1094, 1597, 1495, 1640, 1095, 873, 47, 73, 1672, 321, 318, 1119, 1021, 357, 102, 269, 973, 1619, 66, 1329, 312, 399, 1703, 287, 444, 98, 98, 1595, 306, 1192, 497, 1324, 494]
Wall time: 262 ms


In [30]:
xlsx_file = Path('../IR_Spring2021_ph12_7k.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file)
sheet = wb_obj.active
for doc in sorted_result:
    for row in sheet.iter_rows():
        if row[0].value == 'id':
            continue
        if int(row[0].value) == doc:
            print(f"Document {doc} :\n{row[2].value}", end='\n____________________________________________\n')
#             print(doc)

Document 1596 :
https://www.isna.ir/news/98110100757/مدیران-۴-تیم-آسیایی-ایران-پنجشنبه-با-دبیرکل-AFC-دیدار-می-کنند
____________________________________________
Document 632 :
https://www.isna.ir/news/99080704492/رویارویی-قهرمانان-سابق-لیگ-برتر-فوتبال-در-دیداری-تدارکاتی
____________________________________________
Document 394 :
https://www.isna.ir/news/99053022911/نکونام-بر-دستان-بازیکنان-فولاد
____________________________________________
Document 154 :
https://www.isna.ir/news/99030904913/نظر-موافق-سرمربی-تیم-ملی-فوتسال-با-حذف-پلی-آف-لیگ-برتر
____________________________________________
Document 1583 :
https://www.isna.ir/news/98102720937/جزییات-تصمیم-کمیته-مسابقات-AFC-درباره-ایران-بازی-در-زمین-بی-طرف
____________________________________________
Document 1012 :
https://www.isna.ir/news/99121713154/مجوز-حضور-جودوی-ایران-در-قهرمانی-آسیا-صادر-شد
____________________________________________
Document 667 :
https://www.isna.ir/news/99081912504/رادوشوویچ-امروز-در-محل-تمرین-پرسپولیس-حاضر-شد
_